# API ---->  FOURSQUARE
Hacer requests de:
    - Starbucks
    - Airports
    - Schools
    - Bar
    - Vegan

In [1]:
from src.constants import places
import requests
import pprint
import re
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
import os
import json
import numpy as np

In [2]:
from pymongo import MongoClient

client = MongoClient()

def connectCollection(database, collection):
    ''' return collection from db crunchbase'''
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies_clean')

In [5]:
companies = db.companies_clean.find({ 
    "money_raised":{
        "$gte": 1000000
    }}) 

In [6]:
data_companies = pd.DataFrame(companies)

In [7]:
geo_point = []
for loc in range(len(data_companies)):
    geo_point.append(data_companies.coordinates[loc])  

In [90]:
def near(geo_point, radio_meters=3000):
        geopoint = geo_point
        
        return list(db.companies_clean.find({
        "coordinates.coordinates": {
         "$near": {
             "$geometry": geopoint,
             "$maxDistance": radio_meters
         }}})) 

def numberOffices(data_companies):
    number_offices = []
    list_offices=[]
    for i in range(len(data_companies)):
        num_offices =  near(geo_point[i], radio_meters)
        list_offices.append(num_offices)
        number_offices.append(len(num_offices))
    data_companies['number offices near'] = number_offices
    return data_companies

In [91]:
data_companies = numberOffices(data_companies)

In [95]:
data = data_companies[data_companies['number offices near'] > 1]

In [96]:
data.head()

,_id,name,employees,year,category,id,money_raised,city,country,latitude,longitude,coordinates,number offices near
0,5e3148a26301037b54495dbc,Social Gaming Network,100.0,2011,web,52cdef7c4bab8bd67529831a,17100000.0,Los Angeles,USA,37.446823,-122.161523,"{'type': 'Point', 'coordinates': [-122.161523,...",3
1,5e3148a26301037b54495dbd,Fixya,30.0,2013,web,52cdef7c4bab8bd675297fec,8000000.0,San Mateo,USA,37.566879,-122.323895,"{'type': 'Point', 'coordinates': [-122.323895,...",2
9,5e3148a26301037b54495dc6,Inuvo,50.0,2009,web,52cdef7d4bab8bd67529980b,4200000.0,Clearwater,USA,27.913011,-82.702449,"{'type': 'Point', 'coordinates': [-82.702449, ...",6
10,5e3148a26301037b54495dc8,Burt,30.0,2009,web,52cdef7d4bab8bd6752996f5,3500000.0,Gothenburg,SWE,59.337864,18.036587,"{'type': 'Point', 'coordinates': [18.036587, 5...",2
16,5e3148a26301037b54495dd0,Meez,21.0,2009,web,52cdef7e4bab8bd67529ac2f,10200000.0,San Francisco,USA,37.785271,-122.397582,"{'type': 'Point', 'coordinates': [-122.397582,...",2


In [13]:
data_companies.shape

(22, 13)

In [56]:
data_companies.to_json('./best_companies.json', orient='records',default_handler=str)

In [57]:
data_companies.to_csv('./best_companies.csv') 

In [14]:
geo = []
for coords in data_companies['coordinates']:
    geo.append(coords['coordinates'])

In [59]:
 # Latitud 37
# Longitud -122


In [15]:
def getNearby(lat,lng,query,radius):
    """ Function for queries """
    CLIENT_ID = os.getenv("client_id")
    CLIENT_SECRET = os.getenv("client_secret")
    url = 'https://api.foursquare.com/v2/venues/explore'
    queryParams = dict(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        v='20181023', # DONDE ESTOY
        ll=f"{lat},{lng}", #QUE QUIERO
        query=query, # A QUE DISTANCIA
        radius=radius,
        limit = 25
    )
    res = requests.get(url, params=queryParams)
    return res.json()
  

In [176]:
# QUERY ONLY FOR STOCKHOLM 

ams_coffee= getNearby(52.365758, 4.892872, "starbucks", 2000)

In [16]:
ams = []

try:
    for i in ams_coffee["response"]["groups"][0]["items"]:

        if i["venue"]['location']['city'] != None:
            ams.append({
                "name":i["venue"]['name'],
                "distance": i["venue"]['location']["distance"],
                "country": i["venue"]['location']["country"],
                "city": i["venue"]['location']['city'],
                "latitude": i["venue"]['location']["lat"],
                "longitude": i["venue"]['location']["lng"],
                "coordinates": {
                "type":"Point",
                "coordinates":[i["venue"]['location']["lng"], i["venue"]['location']["lat"]]
                    }})
except:
    print(f"Ha ocurrido algún error.")


Ha ocurrido algún error.


In [97]:
#  Hacer las queries a todas las coordenadas
p = []
for coord in geo:
    for place, value in places.items():
        try:
            request = (getNearby(coord[1],coord[0],value['api'],value['distance']))
            p.append(request)
        except:
            Exception: f"Ha habido problemas con {place}"


In [98]:
def curateResponse(data):
    """ Function for clean request and clean errors """
    places = []
    for item in data:
        try:
            for i in item["response"]["groups"][0]["items"]:
                if i["venue"]['location']['city'] != None:
                    places.append({
                        "name":i["venue"]['name'],
                        "distance": i["venue"]['location']["distance"],
                        "country": i["venue"]['location']["country"],
                        "city": i["venue"]['location']['city'],
                        "latitude": i["venue"]['location']["lat"],
                        "longitude": i["venue"]['location']["lng"],
                        "coordinates": {
                        "type":"Point",
                        "coordinates":[i["venue"]['location']["lng"], i["venue"]['location']["lat"]]
                            }})
        except:
            print(f"Ha ocurrido algún error.")
    return places

In [99]:
info = p+ams

In [61]:
data = pd.DataFrame(curateResponse(info))

Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.
Ha ocurrido algún error.


In [62]:
data.city.unique()

array(['Palo Alto', 'Menlo Park', 'East Palo Alto', 'Redwood City',
       'Mountain View', 'Los Altos', 'San Carlos', 'Stanford',
       'San Mateo', 'Burlingame', 'Foster City', 'Belmont', 'Millbrae',
       'San Bruno', 'South San Francisco', 'San Francisco',
       'Hillsborough', 'New York', 'East Elmhurst', 'Brooklyn', 'Queens',
       'Elmhurst', 'Saint Petersburg', 'Largo', 'Clearwater',
       'Safety Harbor', 'Pinellas Park', 'Belleair Bluffs', 'Tampa',
       'Oldsmar', 'Palm Harbor', 'Stockholm', 'Bromma', 'Solna',
       'Flushing', 'Long Island City', 'Alameda', 'West Palm Beach',
       'Oakland', 'Greater London', 'London', 'Paddington', 'Victoria',
       'Canary Wharf', 'Middlesex', 'stockport', 'Holborn', 'Clerkenwell',
       'Islington', 'Jamaica', 'Secaucus', 'South Bank', 'Jersey City',
       'Santa Monica', 'Los Angeles', 'Daly City', 'Amsterdam',
       'Luchthaven Schiphol', 'Schiphol'], dtype=object)

In [63]:
data.name.value_counts().head()

Starbucks                            360
sweetgreen                            13
Juice Press                           11
Abokado                               11
British Airways Concorde (G-BOAD)      8
Name: name, dtype: int64

In [64]:
data[data.name.str.contains('Schiphol', regex= True)]

,name,distance,country,city,latitude,longitude,coordinates
1644,Platform Schiphol-Oost,8361,Nederland,Schiphol,52.313197,4.805052,"{'type': 'Point', 'coordinates': [4.8050515439..."


In [84]:
#data[data.name.str.contains('Lounge', regex= True)]

In [66]:
# Vamos a organizar los datos, y solo nos quedaremos en la columna 'type' con los valores 
# del diccionario <groups> 
groups= {
    "Coffee Shop": 'Starbucks',
    "Airport":['San Carlos Airport (SQL) (San Carlos Airport)',"Palo Alto Airport (PAO) (Palo Alto Airport)"
                "Santa Monica Airport (SMO) (Santa Monica Airport)","London City Airport (LCY) (London City Airport)",
                "St. Petersburg - Clearwater International Airport (PIE)","Platform Schiphol-Oost",
                "Stockholm-Bromma Airport (BMA) (Stockholm-Bromma Airport)",
                "San Francisco International Airport (SFO) (San Francisco International Airport)",
                "LaGuardia Airport (LGA) (LaGuardia Airport)"],
    "Bar":["Bar","Cast Lounge at Viceroy Santa Monica","DNA Lounge","Nightcub","Cocktail Bar","Pub","Wish Bar & Lounge ","Plunge Rooftop Bar & Lounge","Audio Nightclub","Temple Nightclub",
            "Momofuku Ssäm Bar","Absinthe Brasserie & Bar","Cue Bar","Ritz Bar & Lounge","Friday Night Waltz"],
    "School": ["Annex Stockholm International School","Petite Sorbonne Preschool","Trinity Preschool","Preschool of America","School","MS 297 Middle School",
               "Village Community School","Chelsea School","Public School 3","St. Matthew Catholic School",
              "De School Van Mieke Petiet","Goethe Instituut","Altra College","AltSchool","Sverigefinska skolan i Stockholm"
               "Alta Vista School","Mission Montessori","Step by Step Early Childhood","Rålambshovsskolan"
               "Grace Church School","Greenwich Village School","Santa Monica High School",
               "Stuyvesant High School","Norman Thomas High School","North Shoreview Montessori",
               "Highland Montessori","St. Timothy Catholic School","High Point Elementary School"],
    "Vegan": ["Thai Vegan ","Maoz Vegan" , "Urban Vegan Kitchen", "Golden Era Vegan", 
              "Vegan Junk Food Bar","Garden Fresh Vegan Cuisine ","Elovate Vegan Kitchen","Vegetarian Oasis","Maoz"]
}

def chooseCat(prod):
    for groupName, groupItems in groups.items():
        if prod in groupItems:
            return groupName
    return "OTHER"

In [67]:
data["category"] = data.name.apply(chooseCat)

In [85]:
data[(data.category == 'Bar')].head()

,name,distance,country,city,latitude,longitude,coordinates,category,score
61,Friday Night Waltz,514,United States,Palo Alto,37.449341,-122.156635,"{'type': 'Point', 'coordinates': [-122.1566349...",Bar,3.0
229,Ritz Bar & Lounge,519,United States,New York,40.760844,-73.990322,"{'type': 'Point', 'coordinates': [-73.99032233...",Bar,3.0
706,Temple Nightclub,294,United States,San Francisco,37.787898,-122.397227,"{'type': 'Point', 'coordinates': [-122.3972270...",Bar,2.0
842,Momofuku Ssäm Bar,925,United States,New York,40.731711,-73.985571,"{'type': 'Point', 'coordinates': [-73.98557086...",Bar,4.0
1259,Momofuku Ssäm Bar,493,United States,New York,40.731711,-73.985571,"{'type': 'Point', 'coordinates': [-73.98557086...",Bar,3.0


In [69]:
def selector(row):
    if row['category'] == 'Coffee Shop':
        if row['distance'] <= 200:
            return 1
        elif row['distance'] > 200 and row['distance'] <= 500:
            return 2
        elif row['distance'] > 500 and row['distance'] <= 1000:
            return 3
        elif  row['distance'] > 1000 and row['distance'] <= 2000:
            return 4
        elif  row['distance'] > 2500 and row['distance'] <= 3000:
            return 5
        else:
            return 6
    if row['category'] == 'Airport':
        if row['distance'] <= 5000:
            return 1
        elif  row['distance'] > 5000 and row['distance'] <= 10000:
            return 2
        elif row['distance'] > 10000 and row['distance'] <= 20000:
            return 3
        elif  row['distance'] > 20000 and row['distance'] <= 30000:
            return 4
        elif row['distance'] > 30000:
            return 5
    if row['category'] == 'Vegan':
        if row['distance'] <= 300:
            return 1
        elif  row['distance'] > 300 and row['distance'] <= 500:
            return 2
        elif row['distance'] > 500 and row['distance'] <= 1000:
            return 3
        elif  row['distance'] > 1000 and row['distance'] <= 2000:
            return 4
        elif row['distance'] > 2000:
            return 5
    if row['category'] == 'School':
        if row['distance'] <= 500:
            return 1
        elif  row['distance'] > 500 and row['distance'] <= 1000:
            return 2
        elif row['distance'] > 1000 and row['distance'] <= 1500:
            return 3
        elif  row['distance'] > 2000 and row['distance'] <= 3000:
            return 4
        elif row['distance'] > 3000:
            return 5
    if row['category'] == 'Bar':
        if row['distance'] <= 200:
            return 1
        elif  row['distance'] > 200 and row['distance'] <= 300:
            return 2
        elif row['distance'] > 300 and row['distance'] <= 600:
            return 3
        elif  row['distance'] > 600 and row['distance'] <= 1000:
            return 4
        elif row['distance'] > 1000:
            return 5

In [70]:
data['score'] = data.apply(lambda row : selector(row), axis=1)

In [87]:
data.head()

,name,distance,country,city,latitude,longitude,coordinates,category,score
0,Starbucks,91,United States,Palo Alto,37.446737,-122.160489,"{'type': 'Point', 'coordinates': [-122.1604894...",Coffee Shop,1.0
1,Starbucks,976,United States,Palo Alto,37.443713,-122.171854,"{'type': 'Point', 'coordinates': [-122.1718535...",Coffee Shop,3.0
4,Starbucks,1758,United States,Menlo Park,37.451926,-122.180358,"{'type': 'Point', 'coordinates': [-122.1803583...",Coffee Shop,4.0
61,Friday Night Waltz,514,United States,Palo Alto,37.449341,-122.156635,"{'type': 'Point', 'coordinates': [-122.1566349...",Bar,3.0
70,Starbucks,444,United States,San Mateo,37.562890,-122.324005,"{'type': 'Point', 'coordinates': [-122.324005,...",Coffee Shop,2.0


In [72]:
data.drop(data.loc[data['score']> 4].index, inplace=True)

In [73]:
data.shape

(1664, 9)

In [74]:
data_new = data

In [75]:
data_new.score.value_counts()

3.0    157
2.0     92
1.0     26
4.0     18
Name: score, dtype: int64

In [76]:
data_new.category.value_counts()

OTHER          1371
Coffee Shop     230
School           32
Bar              12
Airport          11
Vegan             8
Name: category, dtype: int64

In [77]:
data_new.drop(data_new.loc[data_new.category == 'OTHER'].index, inplace=True)

In [78]:
data_new = data.reset_index(drop=True)

In [79]:
data_new.country.unique()

array(['United States', 'Sverige', 'United Kingdom', 'Nederland'],
      dtype=object)

In [80]:
data_new.loc[(data_new['country']== 'United States'), 'country'] = 'USA'
data_new.loc[(data_new['country']== 'Sverige'), 'country'] = 'SWE'
data_new.loc[(data_new['country']== 'United Kingdom'), 'country']  = 'GBR'
data_new.loc[(data_new['country']== 'Nederland'), 'country']  = 'NLD'


In [81]:
data_new.columns

Index(['name', 'distance', 'country', 'city', 'latitude', 'longitude',
       'coordinates', 'category', 'score'],
      dtype='object')

In [82]:
data_new = data_new[["name","country","city", "coordinates","longitude","latitude","category", "score"]]

In [83]:
data_new.to_json('./output/queries.json', orient='records',default_handler=str)